In [384]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math

In [385]:
#设置默认值
N=100
M=200
R_D2D=30
R_BS=100
S=12

In [386]:
#设置带宽默认值
#默认单位为MHZ
B_D2D = 20
B_BS = 20

#设置默认用户功率和基站功率
E_D2D = 20
E_BS =20

之后都是func，这里都是默认值

In [387]:
#随机化的local message function
def random_point(car_num, radius):
    x_index = []
    y_index = []
    for i in range(1, car_num + 1):
        theta = random.random() * 2 * np.pi
        r = random.uniform(0, radius)
        x = math.cos(theta) * (r ** 0.5)
        y = math.sin(theta) * (r ** 0.5)
        x_index.append(x)
        y_index.append(y)

    return x_index, y_index

出分布图的逻辑

In [388]:
x,y = random_point(N,R_BS**2)
# plt.title("user terminal distribute version I")
# plt.scatter(x, y, marker='o', label="terminal")
# plt.scatter(0, 0, marker='^', label="base station")
# plt.legend(loc='best')
# #plt.plot(0,0,"^",color="red")
# plt.show()

初始化 用户的五个表
可通行表 长度 N
存储表 长度 M
命中表 长度 M
请求表 长度 M
权重表 长度 3

In [389]:
#得到用户的 第一个表 可通信表 1
# 用户i的表为communication[i,:]
communication = np.zeros((N,N))
for i in range(len(communication)):
    for j in range(len(communication)):
        if (x[i]-x[j])**2+(y[i]-y[j])**2<=R_D2D**2:
            communication[i][j]=1

In [390]:
communication[0].sum()-1

12.0

In [391]:
#初始化用户的存储表 2
cache = np.zeros((N,M))
#这个表会在后续循环中更新

In [392]:
#初始化用户的命中表 3
#这个表是由用户自身的存储表和用户可通行表中其他用户的存储表相加得到的
hit_nums = np.zeros((N,M))

In [393]:
#初始化用户的请求表 4
#是由图神经网络预测得到的，先用随机值代替
rate = 5*np.random.random((N,M))
request = np.zeros((N,M))
for i in range(len(rate)):
    for j in range(len(rate[0])):
        request[i][j]=rate[i][j]/rate[i].sum()


In [394]:
#初始化权重表5
weight = np.zeros((N,3))
weight[:,0]=+1
weight[:,1]=+1

SELF = 0
D2D = 1
BASE = 2

# 1 是 self 的权重，2 是 d2d 的权重  3是 BS的权重
#如果是 命中概率最高， 那么 self =1  d2d =1  BS =0 即可
weight

array([[1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1., 0.],
       [1., 1

开始循环

In [395]:
#循环前一些的值说明

#表示总的效能
U=0
#K表示循环论数 一定是 N的 K次方
K=10
#把每一轮循环的结果 存起来
ans_U = np.zeros(K)
U_self=0
U_D2D = 0

for i in range(K):
    random_user_set = np.arange(0,N,1)
    np.random.shuffle(random_user_set)
    print('这是第的',i*N,'轮迭代')
    #挑选一个用户 index 来是的他的缓存空间存储的对象可以使得 U全局 最大化
    for index in random_user_set:
        cache_U = np.zeros((M,2)) #用户 index 存入文件m 需要一个 表记录
        cache_U_0 = np.zeros((M,2))
        #cache_tmp矩阵表示临时表， index行全1
        cache_tmp =cache.copy()
        cache_tmp[index,:]=np.ones(M)
        cache_tmp_0 = cache.copy()
        cache_tmp_0[index,:]=np.zeros(M)
        #
        hit_nums = np.zeros((N,M))
        hit_nums_0 = np.zeros((N,M))
        for n in range(N):
            if communication[index,n]==0: #过滤所有的不能于index 通信的n
                continue
            for n_nearby in range(N):
                if communication[n,n_nearby]==0:
                    continue
                hit_nums[n,:]+=cache_tmp[n_nearby,:] #得到n所有的hit数量
        for m in range(M):
            cache_U[m,1]=m
            for n in range(N):
                if communication[index,n]==0: #过滤所有的不能于index 通信的n
                    continue
                if hit_nums[n,m]>=1:
                    if cache_tmp[n,m]==1:
                        cache_U[m,0]+=request[n,m]*weight[n,SELF]
                    else:
                        cache_U[m,0]+=request[n,m]*weight[n,D2D]

        for n in range(N):
            if communication[index,n]==0: #过滤所有的不能于index 通信的n
                continue
            for n_nearby in range(N):
                if communication[n,n_nearby]==0:
                    continue
                hit_nums_0[n,:]+=cache_tmp_0[n_nearby,:] #得到n所有的hit数量
        for m in range(M):
            cache_U_0[m,1]=m
            for n in range(N):
                if communication[index,n]==0: #过滤所有的不能于index 通信的n
                    continue
                if hit_nums_0[n,m]>=1:
                    if cache_tmp_0[n,m]==1:
                        cache_U_0[m,0]+=request[n,m]*weight[n,SELF]
                    else:
                        cache_U_0[m,0]+=request[n,m]*weight[n,D2D]

        cache_U[:,0] -= cache_U_0[:,0]

        sort_cache_U=cache_U[np.lexsort(-cache_U[:,::-1].T)]
        cache[index,:]=np.zeros(M)
        for ca in range(S):

            add=sort_cache_U[ca,1]
            cache[index,int(add)]=1
            #print('用户：',index,'  存储下标:',add)


    hit_nums=np.zeros((N,M))
    U_self=0
    U_D2D = 0
    #print(cache.sum()==N*S)
    for n in range(N):
        for n_nearby in range(N):
            if communication[n,n_nearby]==0:
                continue
            hit_nums[n,:]+=cache[n_nearby,:]
            #print(cache[n_nearby].sum())
        #print(hit_nums[n].sum())
    #print(hit_nums.sum())
    for n in range(N):
        for m in range(M):
            if hit_nums[n,m]>=1:
                if cache[n,m]==1:
                    U_self+=request[n,m]*weight[n,SELF]
                else:
                    U_D2D+=request[n,m]*weight[n,D2D]
    print(hit_nums.sum())
    print((U_D2D+U_self)/N)
    print(U_D2D/N,U_self/N)
    ans_U[i]=U/N




这是第的 0 轮迭代
11064.0
0.6159470343418584
0.5461650466505911 0.06978198769126726
这是第的 100 轮迭代
11064.0
0.618628605902261
0.5492803765158724 0.06934822938638857
这是第的 200 轮迭代
11064.0
0.619010074901419
0.5497090518819507 0.06930102301946825
这是第的 300 轮迭代
11064.0
0.6190967661129698
0.5498096274627156 0.0692871386502543
这是第的 400 轮迭代
11064.0
0.6191403929671414
0.5498568470456694 0.06928354592147197
这是第的 500 轮迭代
11064.0
0.6191403929671414
0.5498568470456694 0.06928354592147197
这是第的 600 轮迭代
11064.0
0.6191403929671414
0.5498568470456694 0.06928354592147197
这是第的 700 轮迭代
11064.0
0.6191403929671414
0.5498568470456694 0.06928354592147197
这是第的 800 轮迭代
11064.0
0.6191403929671414
0.5498568470456694 0.06928354592147197
这是第的 900 轮迭代
11064.0
0.6191403929671414
0.5498568470456694 0.06928354592147197


In [396]:
hit_nums.sum()

11064.0

global和self
self是只存自己 请求高的

In [397]:
#self的话，每次用户只会存自己请求概率最高的S个文件
self_cache =  np.zeros((N,M))
for i in range(N):
    request_i=np.zeros((M,2))
    request_i[:,0]=request[i,:].copy().T
    request_i[:,1]=np.arange(0,M,1)
    sort_request_i=request_i[np.lexsort(-request_i[:,::-1].T)]
    for j in range(S):
        self_cache[i,int(sort_request_i[j,1])]=1




In [398]:
hit_nums=np.zeros((N,M))
U_self_self=0
U_self_D2D=0
#print(cache.sum()==N*S)
for n in range(N):
    for n_nearby in range(N):
        if communication[n,n_nearby]==0:
            continue
        hit_nums[n,:]+=self_cache[n_nearby,:]
        #print(cache[n_nearby].sum())
    #print(hit_nums[n].sum())
#print(hit_nums.sum())
for n in range(N):
    for m in range(M):
        if hit_nums[n,m]>=1:
            if self_cache[n,m]==1:
                U_self_self+=request[n,m]*weight[n,SELF]
            else:
                U_self_D2D+=request[n,m]*weight[n,D2D]
print((U_self_self+U_self_D2D)/N)


0.4509277919705487


global分为两种    BS Global 和  World Global

In [399]:
#global仍然是先处理request
#这里的global不只是基站内的所有的用户的喜欢，而是全局 这个数据集的总偏好

BS_rate = np.zeros(M)
for i in range(M):
    BS_rate[i]= rate[:,i].sum()/N

BS_request = np.zeros(M)
for i in range(M):
    BS_request[i]=BS_rate[i]/BS_rate.sum()

world_rate = 5*np.random.random(M)
world_request = np.zeros(M)
for i in range(M):
    world_request[i]=world_rate[i]/world_rate.sum()

# global分为两种



称述下 做全局的应该怎么存？
只存 全局高的文件

In [400]:
BS_cache = np.zeros((N,M))

In [401]:
for i in range(S):
    tmp=np.arange(0,M,1)

    np.random.shuffle(tmp)
    for j in range(N):

        BS_cache[j,tmp[j]]=1

hit_nums=np.zeros((N,M))
U_BS_global_D2D=0
U_BS_global_self=0
#print(cache.sum()==N*S)
for n in range(N):
    for n_nearby in range(N):
        if communication[n,n_nearby]==0:
            continue
        hit_nums[n,:]+=BS_cache[n_nearby,:]
        #print(cache[n_nearby].sum())
    #print(hit_nums[n].sum())
#print(hit_nums.sum())
for n in range(N):
    for m in range(M):
        if hit_nums[n,m]>=1:
            if BS_cache[n,m]==1:
                U_BS_global_self+=request[n,m]*weight[n,SELF]
            else:
                U_BS_global_D2D+=request[n,m]*weight[n,D2D]

print((U_BS_global_self+U_BS_global_D2D)/N)

0.414191651189755


world global的偏好

In [402]:

lr = 1/(N*S)
value = (R_D2D**2)/(R_BS**2)
Alternative = np.zeros(S*N)


def d2d_gd(capacity):
    k = capacity*N
    results = np.zeros(M)
    for i in range(k):
        max=0
        index=0
        for j in range(len(results)):
            if d2d(world_request[j],results[j]+lr)-d2d(world_request[j],results[j])>max:
                max =d2d(world_request[j],results[j]+lr)-d2d(world_request[j],results[j])
                index=j

        Alternative[i]=index
        results[index]+=lr
    return results

def d2d(a,x):
    return a-a*np.exp(-1.0*value* N * x)+a*x*np.exp(-1.0*value* N * x)

result_gd = d2d_gd(S)

In [403]:
Alternative

array([ 49.,   1.,  49., ..., 121.,  93.,  49.])

In [404]:
world_cache=np.zeros((N,M))
for i in range(N):
    for j in range(S):
        tmp=np.random.randint(0,len(Alternative)-1)
        world_cache[i,int(Alternative[tmp])]=1

hit_nums=np.zeros((N,M))
U_world_global_D2D=0
U_world_global_self=0
#print(cache.sum()==N*S)
for n in range(N):
    for n_nearby in range(N):
        if communication[n,n_nearby]==0:
            continue
        hit_nums[n,:]+=world_cache[n_nearby,:]
        #print(cache[n_nearby].sum())
    #print(hit_nums[n].sum())
#print(hit_nums.sum())
for n in range(N):
    for m in range(M):
        if hit_nums[n,m]>=1:
            if world_cache[n,m]==1:
                U_world_global_self+=request[n,m]*weight[n,SELF]
            else:
                U_world_global_D2D+=request[n,m]*weight[n,D2D]

print((U_world_global_self+U_world_global_D2D)/N)

0.20785003811036337


In [405]:
print('个性化：',(U_self+U_D2D)/N)
print('self：',(U_self_D2D+U_self_self)/N)
print('BS global：',(U_BS_global_self+U_BS_global_D2D)/N)
print('World global：',(U_world_global_self+U_world_global_D2D)/N)

个性化： 0.6191403929671414
self： 0.4509277919705487
BS global： 0.414191651189755
World global： 0.20785003811036337
